In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import plot_model
import os

In [2]:
def build_quality_model(input_shape=(128, 128, 3)):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])
    return model


In [3]:
df = pd.DataFrame({
    'bedrooms': np.random.randint(1, 6, 100),
    'bathrooms': np.random.randint(1, 4, 100),
    'solar': np.random.randint(0, 2, 100),
    'AC': np.random.randint(0, 2, 100),
    'pool': np.random.randint(0, 2, 100),
    'avg_nearby_price': np.random.uniform(200000, 800000, 100),
    'quality': np.random.uniform(1, 4, 100),
    'price': np.random.uniform(300000, 900000, 100)
})

In [4]:
X = df.drop(columns=['price']).values
y = df['price'].values

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
price_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])
price_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
price_model.fit(X_scaled, y, epochs=10, batch_size=8)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 366864465920.0000 - mae: 580523.1250
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 331208818688.0000 - mae: 554139.6250
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 330031104000.0000 - mae: 552622.1250
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 351576129536.0000 - mae: 573636.7500
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 335670181888.0000 - mae: 558095.6875
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 349277683712.0000 - mae: 570906.6875
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 335253733376.0000 - mae: 555377.1875
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 362171727872.0000 - mae: 578953.8750
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 377030868992.0000 - mae: 588815.6250
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 333902118912.0000 - mae: 552930.1875  


In [15]:
price_model.save("price_model.h5")

In [7]:
def generate_dummy_nearby_sales(num_properties=6):
    data = {
        'property_id': [f'P-{1000+i}' for i in range(num_properties)],
        'distance_m': np.random.uniform(100, 500, num_properties),
        'bedrooms': np.random.randint(1, 5, num_properties),
        'bathrooms': np.random.randint(1, 3, num_properties),
        'has_solar': np.random.randint(0, 2, num_properties),
        'has_ac': np.random.randint(0, 2, num_properties),
        'has_pool': np.random.randint(0, 2, num_properties),
        'sale_price': np.random.uniform(300000, 800000, num_properties),
        'sale_date': pd.date_range(end=pd.Timestamp.today(), periods=num_properties)
    }
    return pd.DataFrame(data)

In [8]:
dummy_sales_data = generate_dummy_nearby_sales()
avg_nearby_price = dummy_sales_data['sale_price'].mean()

In [9]:
quality_model = build_quality_model()
quality_model.compile(optimizer='adam', loss='mse')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
quality_model.fit(np.random.rand(10, 128, 128, 3), np.random.rand(10), epochs=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.6350


In [16]:
quality_model.save("quality_model.h5")

In [11]:
predicted_quality = np.random.uniform(1, 4)
print(f"\n🖼️ Simulated predicted quality (no image used): {predicted_quality:.2f}")
# if os.path.exists("house1.jpg"):
#     img = load_img("house1.jpg", target_size=(128, 128))
#     img_array = img_to_array(img) / 255.0
#     img_array = np.expand_dims(img_array, axis=0)
#     predicted_quality = quality_model.predict(img_array)[0][0]
# else:
#     print("Image not found❌")
#     predicted_quality = np.random.uniform(1, 4)


🖼️ Simulated predicted quality (no image used): 3.42


In [12]:
features = {
    'bedrooms': 3,
    'bathrooms': 2,
    'solar': 1,
    'AC': 1,
    'pool': 0,
    'avg_nearby_price': avg_nearby_price,
    'quality': predicted_quality
}

In [13]:
input_data = np.array([list(features.values())])
input_scaled = scaler.transform(input_data)

In [18]:
price_model = load_model("price_model.h5", compile=False)
predicted_price = price_model.predict(input_scaled)[0][0]
print(f"\n🏡 Estimated Property Value: ${predicted_price:,.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step

🏡 Estimated Property Value: $12.28
